In [1]:
#install dependencies
!pip install tensorflow==2.3.1 gym keras-rl2 gym[atari]

You should consider upgrading via the '/home/ece/home/ece/anaconda3.1/bin/python3 -m pip install --upgrade pip' command.


In [2]:
#test random environment with OpenAI Gym

In [3]:
import gym
import random

In [4]:
#the code to quit the ROM issue 
!python -m atari_py.import_roms /home/ece/Belgeler/RLimplementation/ROMS

copying zaxxon.bin from /home/ece/Belgeler/RLimplementation/ROMS/Zaxxon (1983) (Coleco) (2454) ~.bin to /home/ece/home/ece/anaconda3.1/lib/python3.8/site-packages/atari_py/atari_roms/zaxxon.bin
copying star_gunner.bin from /home/ece/Belgeler/RLimplementation/ROMS/Stargunner (1983) (Telesys, Alex Leavens) (1005) ~.bin to /home/ece/home/ece/anaconda3.1/lib/python3.8/site-packages/atari_py/atari_roms/star_gunner.bin
copying adventure.bin from /home/ece/Belgeler/RLimplementation/ROMS/Adventure (1980) (Atari, Warren Robinett) (CX2613, CX2613P) (PAL).bin to /home/ece/home/ece/anaconda3.1/lib/python3.8/site-packages/atari_py/atari_roms/adventure.bin
copying tutankham.bin from /home/ece/Belgeler/RLimplementation/ROMS/Tutankham (1983) (Parker Brothers, Dave Engman, Dawn Stockbridge) (PB5340) ~.bin to /home/ece/home/ece/anaconda3.1/lib/python3.8/site-packages/atari_py/atari_roms/tutankham.bin
copying surround.bin from /home/ece/Belgeler/RLimplementation/ROMS/Surround (32 in 1) (Bit Corporation) 

In [5]:
env = gym.make('SpaceInvaders-v0')
height, width, channels = env.observation_space.shape #for image's shape
actions = env.action_space.n

In [6]:
env.unwrapped.get_action_meanings() #actions' names

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [7]:
episodes= 5
for episode in range (1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = random.choice([0,1,2,3,4,5])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print("Episode:{} Score:{}".format(episode, score))
env.close()

Episode:1 Score:345.0
Episode:2 Score:110.0
Episode:3 Score:190.0
Episode:4 Score:50.0
Episode:5 Score:115.0


In [8]:
#create a deep learning model with keras

In [9]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam 

In [22]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [23]:
model = build_model(height, width, channels, actions)

In [24]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 22, 16, 64)     36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 67584)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)               34603520  
_________________________________________________________________
dense_7 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_8 (Dense)              (None, 6)                

In [25]:
#build aget with keras rl

In [26]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [27]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000
                  )
    return dqn

In [28]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-4))

In [ ]:
dqn.fit(env, nb_steps=10000, visualize=False, verbose=2)

Training for 10000 steps ...
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
 1355/10000: episode: 1, duration: 1027.511s, episode steps: 1355, steps per second:   1, episode reward: 765.000, mean reward:  0.565 [ 0.000, 200.000], mean action: 2.532 [0.000, 5.000],  loss: 17.479782, mean_q: 2.410064, mean_eps: 0.894025


In [ ]:
scores = dqn.test(env, nb_episodes=10, visuzalize=True)
print(np.mean(scores.history["episodes_reward"]))

In [ ]:
#reloading agent from memory

In [ ]:
dqn.save_weights("SavedWeights/10k-Fastdqn_weights.h5f")

In [ ]:
del model, dqn

In [ ]:
dqn.load_weights("SavedWeights/1m/dqn_weights.h5f")